
# Tutorial 3: Manipulating header metadata information and saving as .AIM

Author: *Dr. Matthias Walle*
<br>Created: *17 Jan 2024*
<br>Last Modified: *17 Jan 2024*

In this tutorial, we will go through the process of reading, processing, and saving an AIM file (Scanco) using `vtkbone` and `vtk`. We will show 

1) How to manipulate the processing log: This could be useful if you are trying to re-calibrate your images (e.g multicenter studies) 
2) How to manipulate an image using `vtk` and then append the changes made to your processing log
3) How to save the manipulated image again as an AIM file. 

 An AIM file typically includes a pre-header, image structure, processing log, image data, and associated image data, each organized into distinct blocks. The pre-header contains key information about the file's structure, while the image structure data (D3AnyImage) comprises various components like version, type, and dimensions. The processing log records all procedures applied to the image, offering valuable insights into the data processing history. 
 
 Importantly, even when images are processed outside the VMS system we should always record changes and modifications in the processing log for proper documentation. Further, parameters in the processing log can influence the anlaysis of the aim files (e.g. density calibration)
 
 Image data and associated image data are stored in specific formats, often requiring byte order considerations on different platforms. Further, mainly 3 main datatypes exist:

 1) D1Tchar (1 byte): Typically used for masks and segmentations
 2) D1Tshort (2 bytes): Typically used for measurements (native scanner units)
 3) D3float (4 bytes): Typically used for finite element analysis

 

# Import required libraries 

In [1]:
from python_helpers.format_aim_header import logtodict, dicttolog

import numpy as np
import sys
import os
import vtk
import vtkbone

## Setup and Read the Image

First, we'll read an image from a given file. We'll use `vtkbone.vtkboneAIMReader` for this purpose.

In [2]:
input_aim = './data/TRAB_1240.AIM'

print(f'Reading file: {input_aim}')
reader = vtkbone.vtkboneAIMReader()
reader.DataOnCellsOff()
reader.SetFileName(input_aim)
reader.Update()
image = reader.GetOutput()

Reading file: ./data/TRAB_1240.AIM



## Process and Print the Log

We will extract the processing log from the image and then make some modifications to it. Modifications to the calibration equation of the processing log will affect the density evaluation on the VMS/IPL system and could therefore be used to re-calibrate your image data. This could be useful for multicenter studies or to correct for long term scanner drift. 


In [3]:
# Use the GetProcessingLog function to extract the processing log
processing_log = reader.GetProcessingLog()

# We can use this helper function to convert the processing log to a python dictionary
processing_log_dict = logtodict(processing_log)

# Make sure to document that this file has been processed outside the VMS server
processing_log_dict['Created by'] = 'N88_VTKBONE'

# Before you change the calibration equation make sure to save the original data 
processing_log_dict['Original Density: intercept'] = processing_log_dict['Density: intercept']

# Now you can manipulate the your calibration equation. 
# Putting this aim file back to the VMS will change your density analysis
processing_log_dict['Density: intercept'] = -400

# To convert the processing log back to its original format we can use the dicttolog function
print(dicttolog(processing_log_dict))

! Processing Log
!
!-------------------------------------------------------------------------------
Created by                    N88_VTKBONE                                       
Time                          9-APR-2017 15:44:30.45                            
Original file                 dk0:[xtremect2.data.00001240.00005725]d0005707.isq;
Original Creation-Date        20-SEP-2016 13:37:57.73                           
Orig-ISQ-Dim-p                      2304       2304        335
Orig-ISQ-Dim-um                   139852     139852      20334
!-------------------------------------------------------------------------------
Patient Name                  DBQ_153                                           
Index Patient                                    1240
Index Measurement                                5725
!-------------------------------------------------------------------------------
Site                                               21
Scanner ID                                  


## Apply Gaussian Smoothing

Next, we'll apply a Gaussian smoothing filter to the image. This is a common preprocessing step in image analysis.


In [4]:
# Best Practice: Make sure to add to the processing log if you perform changes 
# To your imaging data! The processing log will be appended. This will not affect 
# the function in IPL or on the VMS but only be useful for future users to understand
# your data! 

processing_log_dict['Gauss Filter (sigma)'] = 1.2

# For more information on VTK processing see the VTK documentation! 
gaussian_smooth = vtk.vtkImageGaussianSmooth()
gaussian_smooth.SetInputData(image)
gaussian_smooth.SetStandardDeviation(1.2)  
gaussian_smooth.Update()

# Our output image contains the filtered image
output_image = gaussian_smooth.GetOutput()


## Save the Processed Image

Finally, we will save the processed image and its updated processing log.


In [5]:
# We can now use the VTK bone writer to write our AIM file with the manipulated header to the following path
output_aim = './data/TRAB_1240_MANIPULATED_HEADER.AIM'


# Make sure to write the image in native (scanner) units! See the tutorials for unit conversion!
writer = vtkbone.vtkboneAIMWriter()
writer.SetInputData(output_image)
writer.SetFileName(output_aim)
writer.NewProcessingLogOff() # This needs to be turned off if you provide your own log! 
writer.SetProcessingLog(dicttolog(processing_log_dict))
writer.Update()


## Test the Result

As a final step, let's read back the saved image and print its processing log to ensure everything went as expected.


In [6]:
# Read in our manipulated image
reader = vtkbone.vtkboneAIMReader()
reader.DataOnCellsOff()
reader.SetFileName(output_aim)
reader.Update()
image = reader.GetOutput()

# Check the processing log of our manipulated image
processing_log = reader.GetProcessingLog()
print(processing_log)       

! Processing Log
!
!-------------------------------------------------------------------------------
Created by                    N88_VTKBONE                                       
Time                          9-APR-2017 15:44:30.45                            
Original file                 dk0:[xtremect2.data.00001240.00005725]d0005707.isq;
Original Creation-Date        20-SEP-2016 13:37:57.73                           
Orig-ISQ-Dim-p                      2304       2304        335
Orig-ISQ-Dim-um                   139852     139852      20334
!-------------------------------------------------------------------------------
Patient Name                  DBQ_153                                           
Index Patient                                    1240
Index Measurement                                5725
!-------------------------------------------------------------------------------
Site                                               21
Scanner ID                                  